# Objetivo

Nosso objetivo é procurar por informação relativo a itens essencias

> numpy:
Usar para manipulação aritmética de forma eficiente

> pandas:
Manipulação de arquivos csv, tabelas, estruturas de dados

> duckdb:
Roda os arquivos tudo em memória. Rápido e eficiente.

> parquet:
Armazenamento rápido e eficiente para grandes bases de dados

In [ ]:
!pip install duckdb pandas numpy pyarrow

In [ ]:
!pip install scikit-learn unidecode

In [ ]:
!pip install rapidfuzz

In [ ]:
import pandas as pd
import duckdb as db
import unicodedata
import re
from rapidfuzz import fuzz
from pathlib import Path

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df = pd.read_excel("recursos.xlsx")

In [ ]:
df.to_csv("converted_recursos.csv", index=False)

In [ ]:
data_csv = {
    "disponibilidade1": "disponibilidade-jan-abr-2025.csv",
    "disponibilidade2": "disponibilidade-mai-ago-2025.csv",
    "recursos": "converted_recursos.csv"
}

In [ ]:
for nome, csv_path in data_csv.items():
  parquet_path = Path(csv_path).with_suffix(".parquet")

  # fazer leitura do CSV com pandas
  df = pd.read_csv(csv_path, encoding='latin1') #mudar o enconding

  # Salvar agora como parquet
                #caminho do parquet
  df.to_parquet(parquet_path, engine="pyarrow", compression="zstd")

  # se deu certo vamos ver
  print(f"{csv_path} -> {parquet_path}")

In [ ]:
con = db.connect()

for nome, csv_path in data_csv.items():
  parquet_path = Path(csv_path).with_suffix(".parquet")

  con.execute(f"""
        CREATE OR REPLACE TABLE "{nome}" AS
        SELECT * FROM read_parquet('{parquet_path}')
    """)

In [ ]:
# Exemplo: contar registros em cada tabela
for name in data_csv.keys():
    result = con.execute(f"SELECT COUNT(*) AS total FROM {name}").fetchdf()
    print(name, result)


## 1° Pré-processamento dos dados

Carregar os dados e fazer um pré-processamento para receber eles da melhor forma possível

In [ ]:
path_a = "disponibilidade-jan-abr-2025.parquet"
path_b = "converted_recursos.parquet"

df_a = con.execute(f"SELECT * FROM '{path_a}'").df()
df_b = con.execute(f"SELECT * FROM '{path_b}'").df()


## Vamos normalizar o texto
def norm_text(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str).str.strip().str.upper()
    s = s.map(lambda x: unicodedata.normalize("NFKD", x))
    s = s.str.encode("ascii", "ignore").str.decode("ascii")
    s = s.str.replace(r"[^A-Z0-9 ]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
    return s

## Agora, normalizar o id também
def norm_id(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str).str.strip().str.upper()
    s = s.str.replace(r"[^A-Z0-9]", "", regex=True)
    return s

## normaliza as commas e os pontos
def extrai_km(x: str) -> float | None:
    if not isinstance(x, str): return None
    # pega "KM 123,4" ou "km123.4" etc.
    m = re.search(r'KM\W*([\d]+(?:[.,]\d+)?)', x.upper().replace(',', '.'))
    return float(m.group(1)) if m else None

# padronizações planilha 1 (cadastro/ativos)
df_a["CONC_N"]   = norm_text(df_a.get("Nome_Concessionaria", pd.Series()))
df_a["CONC_ID"]  = norm_id(df_a.get("Cod_Concessionaria", pd.Series()))
df_a["ROD_N"]    = norm_text(df_a.get("Rodovia", pd.Series()))
df_a["EQUIP_N"]  = norm_text(df_a.get("Desc_Componente", pd.Series()))
df_a["NOME_N"]   = norm_text(df_a.get("Nome", pd.Series()))
df_a["MARCA_N"]  = norm_text(df_a.get("Marca", pd.Series()))
df_a["MODELO_N"] = norm_text(df_a.get("Modelo", pd.Series()))
df_a["SENT_N"]   = norm_text(df_a.get("Sentido", pd.Series()))
df_a["IDENT_N"]  = norm_id(df_a.get("Identificacao", pd.Series()))  # candidato a "código do equipamento"
df_a["REGANTT_N"]= norm_id(df_a.get("Cod_Registro_Artesp", pd.Series()))
df_a["EXCED1"]   = df_a.get("Excedente")  # pode ser 0/1, S/N, etc.
df_a["KM"]       = df_a.get("Localizacao", pd.Series()).map(lambda x: extrai_km(x if isinstance(x,str) else ""))

# planilha 2 (medições/ocorrências)
df_b["CONC_N"]   = norm_text(df_b.get("CONCESSIONÁRIA", pd.Series()))
df_b["LOTE_N"]   = norm_text(df_b.get("LOTE", pd.Series()))
df_b["EQUIP_N"]  = norm_text(df_b.get("EQUIPAMENTO", pd.Series()))
df_b["CODEQ_N"]  = norm_id(df_b.get("COD_EQUIPAMENTO", pd.Series()))  # principal candidato a Identificacao/ID
df_b["EXCED2"]   = df_b.get("EXCEDENTE")
df_b["VALOR2"]   = pd.to_numeric(df_b.get("VALOR [%]", pd.Series()).astype(str).str.replace(",", "."), errors="coerce")
# DATA + HORA em timestamp (se existir)
if "DATA" in df_b.columns:
    df_b["DATA_TS"] = pd.to_datetime(df_b["DATA"].astype(str) + " " + df_b.get("HORA","00:00").astype(str), errors="coerce")

## 2° Dicionário de tipos

Vamos tentar alinhar rótulos diferentes para o mesmo equipamento

In [ ]:
# usa nomes diferentes para a mesma coisa (?)
APELIDO_TIPO = {
    "CONTROLADOR SEMAFORICO":"CONTROLADOR",
    "PAINEL DE MENSAGEM VARIAVEL":"PMV",
    "PMV":"PMV",
    "RADAR":"RADAR",
    "LOMBADA ELETRONICA":"RADAR LOMBADA",
    "CAMERA":"CFTV",
    "CFTV":"CFTV",
}

def map_tipo(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str).str.upper()
    return s.map(lambda x: APELIDO_TIPO.get(x, x))

df_a["TIPO_CAN"] = map_tipo(df_a["EQUIP_N"].where(df_a["EQUIP_N"]!="", df_a["NOME_N"]))
df_b["TIPO_CAN"] = map_tipo(df_b["EQUIP_N"])

## 3° Estratégias de combinação

Agora, a partir das variáveis que mais parecem fazer sentido, tentaremos fazer com elas "batam"

- **Match exato**: o código do aparelho é igual em ambas as planilhas (ÓTIMO)
- **Match composto**: mesclaremos diferentes variáveis para tentar chegar em algum lugar

In [ ]:
# MATCH EXATO

cand_codes = []
if "IDENT_N" in df_a and "CODEQ_N" in df_b:
    cand_codes.append(("IDENT_N","CODEQ_N"))
if "REGANTT_N" in df_a and "CODEQ_N" in df_b:
    cand_codes.append(("REGANTT_N","CODEQ_N"))
if "ID" in df_a.columns and "CODEQ_N" in df_b:
    df_a["ID_N"] = norm_id(df_a["ID"])
    cand_codes.append(("ID_N","CODEQ_N"))

matches_exact = []
df_a["_m1"] = False
df_b["_m1"] = False

for a_col,b_col in cand_codes:
    m = df_a.loc[~df_a["_m1"]].merge(
        df_b.loc[~df_b["_m1"]],
        left_on=[a_col,"CONC_N","TIPO_CAN"],  # reforço: mesma empresa e mesmo tipo
        right_on=[b_col,"CONC_N","TIPO_CAN"],
        how="inner",
        suffixes=("_P1","_P2")
    )
    if not m.empty:
        matches_exact.append(m)
        df_a.loc[m.index.get_level_values(0).unique(), "_m1"] = True
        df_b.loc[m.index.get_level_values(1).unique(), "_m1"] = True

df_m1 = pd.concat(matches_exact, ignore_index=True) if matches_exact else pd.DataFrame()